# Import Necessary Libraries

In [1]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
import numpy as np    # for mathematical operations
from tensorflow.keras.preprocessing import image   # for preprocessing the images
from tensorflow.keras.utils import to_categorical
from skimage.transform import resize   # for resizing images
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm

# Creating UCF Training List

Data is sourced from the following link:https://www.crcv.ucf.edu/data/UCF101.php

UCF101 is an action recognition data set of realistic action videos, collected from YouTube, having 101 action categories. This data set is an extension of UCF50 data set which has 50 action categories.

With 13320 videos from 101 action categories, UCF101 gives the largest diversity in terms of actions and with the presence of large variations in camera motion, object appearance and pose, object scale, viewpoint, cluttered background, illumination conditions, etc, it is the most challenging data set to date. As most of the available action recognition data sets are not realistic and are staged by actors, UCF101 aims to encourage further research into action recognition by learning and exploring new realistic action categories.

The videos in 101 action categories are grouped into 25 groups, where each group can consist of 4-7 videos of an action. The videos from the same group may share some common features, such as similar background, similar viewpoint, etc.

Train/Test splits are also sourced from the above links to facilitate training and validation of UCF101 data

In [2]:
f = open(r"C:\Users\Shaun\Documents\GitHub\UCF-101\ucfTrainTestlist\trainlist01.txt")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe having video names
train = pd.DataFrame()
train['video_name'] = videos
train = train[:-1]
train.head()

,video_name
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi 1
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c02.avi 1
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c03.avi 1
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c04.avi 1
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c05.avi 1


In [3]:
f = open(r"C:\Users\Shaun\Documents\GitHub\UCF-101\ucfTrainTestlist\testlist01.txt")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe having video names
test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]
test.head()

,video_name
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01.avi
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c02.avi
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c03.avi
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c04.avi
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c05.avi


# Creating Tags for Training Videos

Creating tags generated from the videonames

In [4]:
# creating tags for training videos
train_video_tag = []
for i in range(train.shape[0]):
    train_video_tag.append(train['video_name'][i].split('/')[0])
    
train['tag'] = train_video_tag
print(train.shape[0])
# creating tags for test videos
test_video_tag = []
for i in range(test.shape[0]):
    test_video_tag.append(test['video_name'][i].split('/')[0])
    
test['tag'] = test_video_tag
print(test)

9537
                                       video_name             tag
0     ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01.avi  ApplyEyeMakeup
1     ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c02.avi  ApplyEyeMakeup
2     ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c03.avi  ApplyEyeMakeup
3     ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c04.avi  ApplyEyeMakeup
4     ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c05.avi  ApplyEyeMakeup
...                                           ...             ...
3778                      YoYo/v_YoYo_g06_c04.avi            YoYo
3779                      YoYo/v_YoYo_g07_c01.avi            YoYo
3780                      YoYo/v_YoYo_g07_c02.avi            YoYo
3781                      YoYo/v_YoYo_g07_c03.avi            YoYo
3782                      YoYo/v_YoYo_g07_c04.avi            YoYo

[3783 rows x 2 columns]


Using OpenCV in order to breakdown videos into frames for training of the model.
This is done by capturing the framerate, and storing this snap shot into images in the "train_1" folder

In [13]:
# storing the frames from training videos
for i in tqdm(range(train.shape[0])):
    count = 0
    videoFile = train['video_name'][i]
    cap = cv2.VideoCapture('UCF-101/'+videoFile.split(' ')[0])   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    x=1
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames in a new folder named train_1
            filename ='train_1/' + videoFile.split('/')[1].split(' ')[0] +"_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()

100%|██████████████████████████████████████████████████████████████████████████████| 9537/9537 [09:06<00:00, 17.44it/s]


Creating a csv file with all the names of the images, this will be used to check against the trained model and to evaluate if the model is accurate as compared to the original labels. 

In [22]:
# getting the names of all the images
images = glob("train_1/*.jpg")
train_image = []
train_class = []
for i in tqdm(range(len(images))):
    # creating the image name
    train_image.append(images[i].split('\\')[1])
    # creating the class of image
    train_class.append(images[i].split('\\')[1].split('_')[1])
    
# storing the images and their class in a dataframe
train_data = pd.DataFrame()
train_data['image'] = train_image
train_data['class'] = train_class

# converting the dataframe into csv file 
train_data.to_csv('train_new.csv',header=True, index=False)

100%|███████████████████████████████████████████████████████████████████████| 73844/73844 [00:00<00:00, 1013018.03it/s]
